In [2]:
import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display

import tensorflow as tf
from tensorflow import keras
from tensorflow.compat import v1
from functools import partial

In [14]:
data = np.load('cqt.npz')
train = librosa.amplitude_to_db(data['spec'], ref=np.max).reshape(-1, 168,87,1) # cqt
train = (train+80)/80
instr_target = data['instr'][:,0] # instr
note_target = data['instr'][:,1] # note
print(train.shape, note_target.shape)
print(train.min(),train.max())

(20488, 168, 87, 1) (20488,)
0.0 1.0


In [6]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(
    train, target, test_size=0.1, stratify=target
)

print(train_data.shape, test_data.shape)

(18439, 168, 87, 1) (2049, 168, 87, 1)


In [ ]:
def U_net_model(input_tensor):
# U-net모델 만들기
## tools
    conv_filters = [16, 32, 64, 128, 256, 512]
    cont_factory = partial(
        keras.layers.Cont, kernel_size=(3,3), strides=1, padding="same"
    )
    cont_act_func = keras.layers.ReLU()
    

## Contracting path
    ## 1
    conv1 = cont_factory(conv_filters[0])(input_tensor)
    batch1 = keras.layers.BatchNormalization(axis=-1)(conv1)
    rel1 = cont_act_func(batch1)
    ## 2
    conv2 = cont_factory(conv_filters[1])(rel1)
    batch2 = keras.layers.BatchNormalization(axis=-1)(conv2)
    rel2 = cont_act_func(batch2)
    ## 3
    conv3 = cont_factory(conv_filters[2])(rel2)
    batch3 = keras.layers.BatchNormalization(axis=-1)(conv3)
    rel3 = cont_act_func(batch3)
    ## 4
    conv4 = cont_factory(conv_filters[3])(rel3)
    batch4 = keras.layers.BatchNormalization(axis=-1)(conv4)
    rel4 = cont_act_func(batch4)
    ## 5
    conv5 = cont_factory(conv_filters[4])(rel4)
    batch5 = keras.layers.BatchNormalization(axis=-1)(conv5)
    rel5 = cont_act_func(batch5)
    ## 6
    conv6 = cont_factory(conv_filters[5])(rel5)

## Expansive path